In [ ]:
%pip install qiskit ipywidgets
%pip install matplotlib
%pip install qiskit-ibm-runtime
%pip install pylatexenc
%pip install qiskit-ionq


In [ ]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble, Aer
from qiskit.circuit.library import NLocal
from qiskit.primitives import Sampler as SamplerPRIM

import inspect
f = open("C:\\Users\\fouad\\Documents\\tokens.txt", "r")

#IBM Backend
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler as SamplerIBM, Session, Options, Estimator
next = f.readline()
# Put your own token here if you would like to run it
TOKEN_IBM = next[:len(next)-1]
service_ibm = QiskitRuntimeService(channel="ibm_quantum", token=TOKEN_IBM)
backend_ibm = service_ibm.backend("ibmq_qasm_simulator")

#AER Backend and Noise
from qiskit_aer.backends import AerSimulator
from qiskit_aer.noise import NoiseModel

#IonQ Backend
from qiskit_ionq import IonQProvider, ErrorMitigation

next = f.readline()
# Put your own token here if you would like to run it
TOKEN_IONQ = next[:len(next)-1]
service_ionq = IonQProvider(TOKEN_IONQ)
backend_ionq = service_ionq.get_backend("ionq_simulator")

f.close()

import math
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
%pip show qiskit-ionq
%pip show qiskit

In [ ]:
# Building the testing circuit
BITS = 4
SHOTS = 1000
qreg = QuantumRegister(BITS)
creg = ClassicalRegister(BITS)
qc = QuantumCircuit(qreg, creg)

for i in range(BITS):
    qc.h(i)
    qc.measure(i,i)

display(qc.draw("mpl"))

In [ ]:
# Using Sampler offerred by IBM
sampler = SamplerIBM(backend_ibm)
job = sampler.run(qc, shots=SHOTS)
result = job.result().quasi_dists[0]
plt.bar(result.keys(), result.values())
plt.title("IBM Sampler, with " + str(SHOTS) + " shots. Average: " + str(sum(result.values())/len(result.values())))
plt.show()

# Using default Sampler primitive
sampler = SamplerPRIM()
job = sampler.run(qc,shots=SHOTS)
result = job.result().quasi_dists[0]
plt.bar(result.keys(),result.values())
plt.title("Default Sampler, with " + str(SHOTS) + " shots. Average: " + str(sum(result.values())/len(result.values())))
plt.show()

# This is to compare the difference between the IonQ Sampler, which samples using real probabilities, and the default sampler, 

In [ ]:
# Testing IonQ Backend. Does not have error mitigation (options are unused)

job = backend_ionq.run(qc)
counts = job.result().get_counts()
plt.bar(counts.keys(), counts.values())
plt.title("IonQ Simulator, without Error Mitigation. Average: " + str(sum(counts.values())/len(counts.values())))
plt.show()


In [ ]:
from qiskit import QuantumCircuit
from qiskit_ionq import ErrorMitigation

# Create a basic Bell State circuit:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])
display(qc.draw())

times = 10

avgs_noise = []
avgs_sharp = []
for i in range(times):
    # Run the circuit on IonQ's platform with error mitigation:
    job = backend_ionq.run(qc, error_mitigation=ErrorMitigation.DEBIASING)

    # Print the results.
    avg= sum(job.result().get_counts().values())/len(job.result().get_counts().values())
    avgs_noise += [avg]

    # Get results with a different aggregation method when debiasing
    # is applied as an error mitigation strategy
    avg = sum(job.result(sharpen=True).get_counts().values())/len(job.result(sharpen=True).get_counts().values())
    avgs_sharp += [avg]


total_avg_noise = sum(avgs_noise)/len(avgs_noise)
total_avg_sharp = sum(avgs_sharp)/len(avgs_sharp)
print("Total average, noisy: " + str(total_avg_noise))
print("Total average, sharpened: " + str(total_avg_sharp))



In [ ]:
#Testing IBM Sampler, with default resilience level of 1, which uses M3 error mitigation

options_ibm = Options()
options_ibm.resilience_level = 1
with Session(service=service_ibm, backend="ibmq_qasm_simulator") as session:
    sampler = SamplerIBM(session=session, options=options_ibm)

job = sampler.run(qc)
result = job.result()
plt.bar(counts.keys(), counts.values())
plt.title("IBM Sampler, with M3 Mitigation. Average: " + str(sum(counts.values())/len(counts.values())))
plt.show()

In [ ]:
#Testing different resilience levels for the Estimator of QiskitRuntimeService

from qiskit.circuit import Parameter, QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

theta = Parameter("theta")

qc = QuantumCircuit(2)
qc.x(1)
qc.h(0)
qc.cp(theta, 0, 1)
qc.h(0)
observables = SparsePauliOp.from_list([("ZZ", 1)])

qc.draw("mpl")

## Setup phases
import numpy as np

phases = np.linspace(0, 2 * np.pi, 50)

# phases need to be expressed as a list of lists in order to work
individual_phases = [ [phase] for phase in phases]
## Create noise model
from qiskit.providers.fake_provider import FakeManila
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import Options


# Make a noise model
fake_backend = FakeManila()
noise_model = NoiseModel.from_backend(fake_backend)

# Set options to include the noise model
options = Options()
options.simulator = {
    "noise_model": noise_model,
    "basis_gates": fake_backend.configuration().basis_gates,
    "coupling_map": fake_backend.configuration().coupling_map,
    "seed_simulator": 42,
}

from qiskit_ibm_runtime import Session, QiskitRuntimeService, Estimator

# Add your token below
service = QiskitRuntimeService(channel="ibm_quantum", token=TOKEN_IBM)

noisy_exp_values = []
exp_values_with_em_es = []

r_level = 1

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    # generate noisy results
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=[qc] * len(phases),
        parameter_values=individual_phases,
        observables=[observables] * len(phases),
        shots=1000,
    )
    result = job.result()
    noisy_exp_values = result.values

    options.optimization_level = 2
    options.resilience_level = r_level

    # leverage mitigation and suppression
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=[qc] * len(phases),
        parameter_values=individual_phases,
        observables=[observables] * len(phases),
        shots=1000,
    )
    result = job.result()
    exp_values_with_em_es = result.values

    session.close()



import matplotlib.pyplot as plt
plt.plot(phases, noisy_exp_values, "o", label="Noisy")
plt.plot(phases, exp_values_with_em_es, "o", label="Mitigation + Suppression")
plt.plot(phases, 2 * np.sin(phases / 2) ** 2 - 1, label="Theory")
plt.xlabel("Phase")
plt.ylabel("Expectation")
plt.title("Estimator with Resilience Level " + str(r_level))
plt.legend()
plt.show()